## Chapter3 手写数字识别

### 书中的源码已fork
https://github.com/wangyake/deep-learning-from-scratch/blob/master/ch03/mnist_show.py

* mnist_show.py 在ch03文件夹下
* 包含load_mnist()函数的mnist.py文件在dataset文件夹下

### 读源码可得
* 数据地址： http://yann.lecun.com/exdb/mnist/
* 60，000 training examples
* 10，000 test examples
* subset of NIST

### 另外的途径
```
>>>from tensorflow.examples.tutorials.mnist import input_data
>>>mnist = input_data.read_data_sets("./")

>>>print(mnist.train.images.shape)
>>>print(mnist.train.labels.shape)

>>>print(mnist.validation.images.shape)
>>>print(mnist.validation.labels.shape)

>>>print(mnist.test.images.shape)
>>>print(mnist.test.labels.shape)
```

In [8]:
# coding: utf-8
try:
    import urllib.request   # 下载
except ImportError:
    raise ImportError('You should use Python 3.x')
import os.path
import gzip
import pickle
import os
import numpy as np

In [9]:
url_base = 'http://yann.lecun.com/exdb/mnist/'
key_file = {
    'train_img':'train-images-idx3-ubyte.gz',
    'train_label':'train-labels-idx1-ubyte.gz',
    'test_img':'t10k-images-idx3-ubyte.gz',
    'test_label':'t10k-labels-idx1-ubyte.gz'
}

In [11]:
dataset_dir = '/Users/wangyake/Documents/机器学习/笔记/深度学习入门'
save_file = dataset_dir + "/mnist.pkl"

train_num = 60000
test_num = 10000
img_dim = (1, 28, 28)   # label img X：Y轴
img_size = 784

In [12]:
# 下载单个文件
def _download(file_name):
    file_path = dataset_dir + "/" + file_name

    if os.path.exists(file_path):
        return

    print("Downloading " + file_name + " ... ")
    urllib.request.urlretrieve(url_base + file_name, file_path)
    print("Done")

In [13]:
# 下载mnist数据集的4个数据文件
def download_mnist():
    for v in key_file.values():
       _download(v)

In [14]:
# 读入label
def _load_label(file_name):
    file_path = dataset_dir + "/" + file_name

    print("Converting " + file_name + " to NumPy Array ...")
    with gzip.open(file_path, 'rb') as f:
            labels = np.frombuffer(f.read(), np.uint8, offset=8)
    print("Done")

    return labels

In [15]:
# 读入图片
def _load_img(file_name):
    file_path = dataset_dir + "/" + file_name

    print("Converting " + file_name + " to NumPy Array ...")
    with gzip.open(file_path, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=16)
    data = data.reshape(-1, img_size)
    print("Done")

    return data

In [ ]:
# 解压读取数据到字典dataset
def _convert_numpy():
    dataset = {}
    dataset['train_img'] =  _load_img(key_file['train_img'])
    dataset['train_label'] = _load_label(key_file['train_label'])
    dataset['test_img'] = _load_img(key_file['test_img'])
    dataset['test_label'] = _load_label(key_file['test_label'])

    return dataset

In [16]:
'''
数据集初始化
1.下载
2.解压读取到字典dataset
3.dataset写入pkl文件以便以后快速读取

* with open包含了文件对象的关闭
* 读取文件参数
    rb   读取二进制文件
    w    写
''' 

def init_mnist():
    download_mnist()
    dataset = _convert_numpy()
    print("Creating pickle file ...")
    with open(save_file, 'wb') as f:
        pickle.dump(dataset, f, -1)
    print("Done!")

In [17]:
# 对label进行one_hot编码
def _change_one_hot_label(X):
    T = np.zeros((X.size, 10))   # 数据条数*10的矩阵
    for idx, row in enumerate(T):  # idx：0~数据条数  ，row： 第idx条数据
        row[X[idx]] = 1        # 例如：idx为18，则X[18]为第18个数字（0~9），假设为7。则idx 18数据row[0,0,0,0,0,0,0,1,0,0]

    return T

In [18]:
def load_mnist(normalize=True, flatten=True, one_hot_label=False):
    if not os.path.exists(save_file):
        init_mnist()

    with open(save_file, 'rb') as f:
        dataset = pickle.load(f)
    
    # 归一化
    if normalize:
        for key in ('train_img', 'test_img'):
            dataset[key] = dataset[key].astype(np.float32)
            dataset[key] /= 255.0

    if one_hot_label:
        dataset['train_label'] = _change_one_hot_label(dataset['train_label'])
        dataset['test_label'] = _change_one_hot_label(dataset['test_label'])
    
    # 转化为(1,784)
    if not flatten:
         for key in ('train_img', 'test_img'):
            dataset[key] = dataset[key].reshape(-1, 1, 28, 28)

    return (dataset['train_img'], dataset['train_label']), (dataset['test_img'], dataset['test_label'])

In [20]:
# coding: utf-8
import sys, os
# 这里不需要，下载到笔记同目录下
# sys.path.append(os.pardir)  
import numpy as np
# 这里不需要，上面给出（未分模块）
# from dataset.mnist import load_mnist 
from PIL import Image


def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

img = x_train[0]
label = t_train[0]
print(label)  # 5

print(img.shape)  # (784,)
img = img.reshape(28, 28)  # 转化为图片矩阵28*28
print(img.shape)  # (28, 28)

img_show(img)

Done
Done
Done
Done
Converting train-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting train-labels-idx1-ubyte.gz to NumPy Array ...
Done
Converting t10k-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting t10k-labels-idx1-ubyte.gz to NumPy Array ...
Done
Creating pickle file ...
Done!
5
(784,)
(28, 28)


## 神经网络代码实现

In [21]:
# 因为不做training只做test，所以只读取test数据
def get_data():
    (x_train, t_train), (x_test, t_test) = \
        load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test

In [23]:
def init_network():
    with open('sample_weight.pkl', 'rb') as f:
        network = pickle.load(f)
    
    return network

In [41]:
# 查看weight
weight = init_network()
print(weight.keys())

# 查看数据
x, _ = get_data()
network = init_network()
W1, W2, W3 = network['W1'], network['W2'], network['W3']

print(x.shape)
print(x[0].shape)
print(W1.shape)
print(W2.shape)
print(W3.shape)


dict_keys(['b2', 'W1', 'b1', 'W2', 'W3', 'b3'])
(10000, 784)
(784,)
(784, 50)
(50, 100)
(100, 10)


In [30]:
def sigmoid(a):
    return 1/(1+np.exp(-a))

0.5


In [34]:
def softmax(a):
    exp_a = np.exp(a)
    exp_sum = np.sum(exp_a)
    return exp_a/exp_sum   

In [35]:
def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)
    
    return y

In [38]:
x, t = get_data()  # x_test, t_test
network = init_network()

accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y)  # 概率最高的元素的索引
    if p == t[i]:
        accuracy_cnt += 1

print('Accuracy:' + str(float(accuracy_cnt) / len(x)))    

Accuracy: 0.9352


## 批处理


In [45]:
# 测试
for i in range(0, 10, 2):   # 不包含10
    print(i)

0
2
4
6
8


In [46]:
x, t = get_data()  # x_test, t_test
network = init_network()

batch_size = 100
accuracy_cnt = 0
for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)    #axis=0 为行，1为列
    accuracy_cnt += np.sum(p == t[i:i+batch_size])
    
print('Accuracy:' + str(float(accuracy_cnt) / len(x)))  

Accuracy:0.9352
